In [1]:
!pip install pytesseract pillow


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\vedaa\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
from PIL import Image
import pytesseract

# path to your tesseract installation
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# open the image
img = Image.open('receipt.jpg')

# extract text
text = pytesseract.image_to_string(img)
print(text)


Address Lorem Ipsum 8/24
Date:

Manager Lorem Ipsum
Description Price
Orange Juice $2.15
Apples $3.50
Tomato $2.40
Fish 36.99
Beef $10.00
Onion $1.25
Cheese $3.40
Tax

TOTAL $29.69

THANK YOU

123456778963578021



In [7]:
import csv
import os

# ---------- User Settings ----------
principal_amount = 1000.0  # Your available principal
investments = [             # Investment options: (name, minimum required)
    ("Stock Market", 500),
    ("Mutual Funds", 200),
    ("Fixed Deposit", 100),
    ("Bonds", 150),
    ("Crypto", 50)
]

# ---------- Step 1: Read receipt_data.csv and calculate total expenses ----------
receipt_file = 'receipt_data.csv'
total_expenses = 0.0

if not os.path.exists(receipt_file):
    print(f"❌ {receipt_file} not found. Make sure the CSV exists.")
    exit()

with open(receipt_file, 'r') as f:
    reader = csv.DictReader(f)
    for row in reader:
        try:
            total_expenses += float(row['Price'])
        except ValueError:
            pass  # skip invalid prices

# ---------- Step 2: Calculate remaining amount ----------
remaining_amount = principal_amount - total_expenses

# ---------- Step 3: Suggest affordable investments ----------
suggested_investments = [inv[0] for inv in investments if remaining_amount >= inv[1]]
suggested_investments_str = ", ".join(suggested_investments) if suggested_investments else "No suitable investment"

# ---------- Step 4: Write/Update expenses.csv ----------
expenses_file = 'expenses.csv'

with open(expenses_file, 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(["Total Expenses", "Principal Amount", "Remaining Amount", "Suggested Investment"])
    writer.writerow([total_expenses, principal_amount, remaining_amount, suggested_investments_str])

print(f"✅ {expenses_file} updated successfully!")
print(f"Total Expenses: ${total_expenses:.2f}")
print(f"Remaining Amount: ${remaining_amount:.2f}")
print(f"Suggested Investments: {suggested_investments_str}")


✅ expenses.csv updated successfully!
Total Expenses: $89.38
Remaining Amount: $910.62
Suggested Investments: Stock Market, Mutual Funds, Fixed Deposit, Bonds, Crypto


In [15]:
import pandas as pd
import yfinance as yf
from sklearn.ensemble import RandomForestRegressor
import os

# ------------------- USER SETTINGS -------------------
STOCKS = ["AAPL", "MSFT", "GOOGL", "AMZN"]  # Stocks to consider
EXPENSES_FILE = "expenses.csv"  # File storing balance
RECOMMENDATION_FILE = "stock_recommendation.csv"

# ------------------- STEP 1: Read Balance -------------------
if not os.path.exists(EXPENSES_FILE):
    print(f"❌ {EXPENSES_FILE} not found. Please create it with your principal amount.")
    exit()

expenses_df = pd.read_csv(EXPENSES_FILE)
# Assuming the CSV has columns: Total Expenses, Principal Amount, Remaining Amount
balance = float(expenses_df['Remaining Amount'].iloc[-1])
print(f"💰 Current Balance: ${balance:.2f}")

# ------------------- STEP 2: Fetch Stock Data -------------------
stock_data = []
for symbol in STOCKS:
    try:
        ticker = yf.Ticker(symbol)
        hist = ticker.history(period="5d")  # last 5 days
        latest_close = hist['Close'][-1]
        # Simple features
        volatility = hist['Close'].pct_change().std()  # standard deviation of returns
        momentum = (hist['Close'][-1] - hist['Close'][0]) / hist['Close'][0]  # 5-day momentum

        stock_data.append({
            "symbol": symbol,
            "price": latest_close,
            "volatility": volatility,
            "momentum": momentum
        })
    except Exception as e:
        print(f"❌ Error fetching {symbol}: {e}")

df = pd.DataFrame(stock_data)
if df.empty:
    print("❌ No stock data fetched.")
    exit()

# ------------------- STEP 3: Train ML Model (Simulated) -------------------
# For prototype, we simulate training with dummy target: buy as many shares as possible within balance/len(stocks)
# Minimum 1 share for affordable stocks
df['target_shares'] = (balance / len(df) / df['price']).apply(lambda x: max(int(x), 0))


features = ['price', 'volatility', 'momentum']
X = df[features]
y = df['target_shares']

model = RandomForestRegressor()
model.fit(X, y)

# ------------------- STEP 4: Predict Shares to Buy -------------------
df['shares_to_buy'] = model.predict(X).astype(int)

# Ensure we do not exceed balance
df['total_cost'] = df['shares_to_buy'] * df['price']
total_allocation = df['total_cost'].sum()

if total_allocation > balance:
    # Scale down proportionally
    scale = balance / total_allocation
    df['shares_to_buy'] = (df['shares_to_buy'] * scale).astype(int)
    df['total_cost'] = df['shares_to_buy'] * df['price']

# ------------------- STEP 5: Save Recommendations -------------------
df[['symbol', 'price', 'shares_to_buy', 'total_cost']].to_csv(RECOMMENDATION_FILE, index=False)
print(f"✅ Stock recommendation saved to {RECOMMENDATION_FILE}")
print(df[['symbol', 'price', 'shares_to_buy', 'total_cost']])



💰 Current Balance: $1499.75


C:\Users\vedaa\AppData\Local\Temp\ipykernel_36664\13698603.py:27: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  latest_close = hist['Close'][-1]
C:\Users\vedaa\AppData\Local\Temp\ipykernel_36664\13698603.py:30: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  momentum = (hist['Close'][-1] - hist['Close'][0]) / hist['Close'][0]  # 5-day momentum
C:\Users\vedaa\AppData\Local\Temp\ipykernel_36664\13698603.py:27: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, us

✅ Stock recommendation saved to stock_recommendation.csv
  symbol       price  shares_to_buy  total_cost
0   AAPL  247.449997              0         0.0
1   MSFT  511.609985              0         0.0
2  GOOGL  251.460007              0         0.0
3   AMZN  214.470001              0         0.0


C:\Users\vedaa\AppData\Local\Temp\ipykernel_36664\13698603.py:27: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  latest_close = hist['Close'][-1]
C:\Users\vedaa\AppData\Local\Temp\ipykernel_36664\13698603.py:30: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  momentum = (hist['Close'][-1] - hist['Close'][0]) / hist['Close'][0]  # 5-day momentum


In [20]:
import pandas as pd
import yfinance as yf
from sklearn.ensemble import RandomForestRegressor
import numpy as np
import os

# ------------------- USER SETTINGS -------------------
STOCKS = ["AAPL", "MSFT", "GOOGL", "AMZN"]
EXPENSES_FILE = "expenses.csv"  # CSV with Total Expenses, Principal Amount, Remaining Amount
RECOMMENDATION_FILE = "stock_recommendation.csv"
DAYS_HISTORY = 60  # Use past 60 days for features
ALLOCATION_STRATEGY = "equal_weight"  # Could be "momentum", "risk_adjusted"

# ------------------- STEP 1: Read Balance from CSV -------------------
if not os.path.exists(EXPENSES_FILE):
    print(f"❌ {EXPENSES_FILE} not found. Please create it with your principal amount.")
    exit()

expenses_df = pd.read_csv(EXPENSES_FILE)
balance = float(expenses_df['Remaining Amount'].iloc[-1])
print(f"💰 Current Balance: ${balance:.2f}")

# ------------------- STEP 2: Fetch Historical Data & Generate Training Data -------------------
all_data = []

for symbol in STOCKS:
    ticker = yf.Ticker(symbol)
    hist = ticker.history(period=f"{DAYS_HISTORY}d")
    if hist.empty:
        continue
    
    # Compute features
    hist['volatility'] = hist['Close'].pct_change().rolling(5).std()
    hist['momentum'] = (hist['Close'] - hist['Close'].shift(5)) / hist['Close'].shift(5)
    hist['price'] = hist['Close']
    
    hist = hist.dropna(subset=['volatility', 'momentum'])
    
    # Generate target shares based on rule-based allocation
    if ALLOCATION_STRATEGY == "equal_weight":
        allocation = balance / len(STOCKS)
        hist['target_shares'] = (allocation / hist['Close']).apply(np.floor)
    elif ALLOCATION_STRATEGY == "momentum":
        hist['score'] = hist['momentum'].clip(lower=0)
        total_score = hist['score'].sum()
        hist['target_shares'] = ((hist['score'] / total_score) * balance / hist['Close']).apply(np.floor)
    
    hist['symbol'] = symbol
    all_data.append(hist[['symbol', 'price', 'volatility', 'momentum', 'target_shares']])

df = pd.concat(all_data, ignore_index=True)
df = df[df['target_shares'] > 0]  # Remove rows with zero shares

# ------------------- STEP 3: Train ML Model -------------------
features = ['price', 'volatility', 'momentum']
X = df[features]
y = df['target_shares']

model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X, y)

# ------------------- STEP 4: Predict Latest Shares -------------------
latest_data = []

for symbol in STOCKS:
    ticker = yf.Ticker(symbol)
    hist = ticker.history(period="5d")
    if hist.empty:
        continue
    price = hist['Close'].iloc[-1]
    volatility = hist['Close'].pct_change().std()
    momentum = (hist['Close'].iloc[-1] - hist['Close'].iloc[0]) / hist['Close'].iloc[0]
    latest_data.append({'symbol': symbol, 'price': price, 'volatility': volatility, 'momentum': momentum})

latest_df = pd.DataFrame(latest_data)
latest_df['shares_to_buy'] = model.predict(latest_df[features]).astype(int)
latest_df['total_cost'] = latest_df['shares_to_buy'] * latest_df['price']

# ------------------- STEP 5: Scale to Balance if Needed -------------------
total_allocation = latest_df['total_cost'].sum()
if total_allocation > balance:
    scale = balance / total_allocation
    latest_df['shares_to_buy'] = (latest_df['shares_to_buy'] * scale).astype(int)
    latest_df['total_cost'] = latest_df['shares_to_buy'] * latest_df['price']

# ------------------- STEP 6: Save Recommendations -------------------
latest_df[['symbol', 'price', 'shares_to_buy', 'total_cost']].to_csv(RECOMMENDATION_FILE, index=False)
print(f"✅ Stock recommendation saved to {RECOMMENDATION_FILE}")
print(latest_df[['symbol', 'price', 'shares_to_buy', 'total_cost']])


💰 Current Balance: $1000.00
✅ Stock recommendation saved to stock_recommendation.csv
  symbol       price  shares_to_buy  total_cost
0   AAPL  247.449997              0         0.0
1   MSFT  511.609985              0         0.0
2  GOOGL  251.460007              0         0.0
3   AMZN  214.470001              0         0.0
